In [2]:
import findspark
findspark.init()
import pyspark

In [75]:
import pandas as pd
import numpy as np

from pyspark.sql import *
from pyspark.sql.types import *

from pyspark.sql.functions import explode
from pyspark.sql.functions import split

from pyspark.sql.functions import udf
from pyspark.sql.functions import col

import matplotlib.pyplot as plt

from geopy.geocoders import Nominatim
from geopy import distance
from babel import Locale

In [4]:
import plotly as pyy
import plotly.plotly as py
import plotly.graph_objs as go
pyy.tools.set_credentials_file(username='frederic.bischoff',api_key='TEQ3zcIlRzzdVP08naSo')

In [5]:
#to generate colors
import seaborn as sns
sns.set()

In [6]:
spark = SparkSession.builder.getOrCreate()

In [7]:
import json
import folium
import branca.colormap as cm

In [8]:
from pyspark.sql.functions import initcap

# Openfoodfact data 

In [9]:
#We load the entire OpenFoodFacts file

df_data = spark.read.option("sep", "\t").option("header", True).csv('./data/data.csv')

In [10]:
#The title of the features
df_data

DataFrame[code: string, url: string, creator: string, created_t: string, created_datetime: string, last_modified_t: string, last_modified_datetime: string, product_name: string, generic_name: string, quantity: string, packaging: string, packaging_tags: string, brands: string, brands_tags: string, categories: string, categories_tags: string, categories_en: string, origins: string, origins_tags: string, manufacturing_places: string, manufacturing_places_tags: string, labels: string, labels_tags: string, labels_en: string, emb_codes: string, emb_codes_tags: string, first_packaging_code_geo: string, cities: string, cities_tags: string, purchase_places: string, stores: string, countries: string, countries_tags: string, countries_en: string, ingredients_text: string, allergens: string, allergens_en: string, traces: string, traces_tags: string, traces_en: string, serving_size: string, serving_quantity: string, no_nutriments: string, additives_n: string, additives: string, additives_tags: stri

# Part 1: Nutri-Score
### Which countries have the healthiest alimentation? 

We check which countries are the most represented in the datafile.

Before manipulating the data to exploit themn we wanted to know which country have the most product on OpenFactFood, according to this results we will decide on which country we will focus for the analysis.

In [11]:
#We split the countries_tags column to have single country in each row.
#Then we count the occurrence of each country, and order them from the bigger to the smaller count.

countries = df_data.select(explode(split('countries_tags', ','))).withColumnRenamed("col", "countries_tags")
#countries.show(200)

counted_countries = countries.groupBy('countries_tags').count().sort("count", ascending = False)
counted_countries.show(40)

+-------------------+------+
|     countries_tags| count|
+-------------------+------+
|          en:france|446394|
|   en:united-states|175478|
|         en:germany| 19489|
|     en:switzerland| 19365|
|         en:belgium|  9735|
|           en:spain|  8341|
|  en:united-kingdom|  8086|
|          en:canada|  3805|
|           en:italy|  3035|
|       en:australia|  2694|
|          en:mexico|  2360|
|          en:russia|  2203|
|     en:netherlands|  1548|
|          en:sweden|  1526|
|        en:portugal|  1441|
|         en:denmark|   949|
|         en:austria|   928|
|         en:romania|   799|
|  en:czech-republic|   646|
|         en:hungary|   643|
|      en:luxembourg|   613|
|          en:serbia|   595|
|         en:ireland|   588|
|         en:reunion|   552|
|          en:brazil|   529|
|         en:morocco|   501|
|           en:japan|   493|
|         en:algeria|   491|
|      en:guadeloupe|   484|
|              en:en|   477|
|          en:poland|   435|
|        en:th

In [12]:
df_countries = counted_countries.toPandas() 

Then, we wanted to see the most represented categories, to be able to treat not only the entire dataset, but also interesting sub-datasets.

In [13]:
#We split the categories_tags column to have single categoriy in each row.
#Then we count the occurrence of each category, and order them from the bigger to the smaller count.

categories = df_data.select(explode(split('categories_tags', ','))).withColumnRenamed("col", "categories_tags")
#categories.show(200, False)

counted_categories = categories.groupBy('categories_tags').count().sort("count", ascending = False)
counted_categories.show(20, False)

+------------------------------------+-----+
|categories_tags                     |count|
+------------------------------------+-----+
|en:plant-based-foods-and-beverages  |51261|
|en:plant-based-foods                |43981|
|en:beverages                        |29798|
|en:sugary-snacks                    |26277|
|en:dairies                          |18477|
|en:non-alcoholic-beverages          |16523|
|en:meals                            |15601|
|en:fruits-and-vegetables-based-foods|15362|
|en:cereals-and-potatoes             |15324|
|en:fermented-foods                  |12164|
|en:fermented-milk-products          |12127|
|en:biscuits-and-cakes               |11895|
|en:spreads                          |10928|
|en:cereals-and-their-products       |9792 |
|en:meats                            |9686 |
|en:groceries                        |9583 |
|en:breakfasts                       |9547 |
|en:desserts                         |9091 |
|en:plant-based-beverages            |8633 |
|en:fruits

## A/ General overview of the top 20 countries alimentation

Here we will focus on the top 20 countries with the most products, that means, according to the table of last part, that we will study countries from France with 446394 products to czech-republic with 646 products.


In [14]:
def NutriScoreCountry(country_tag_name):
    df_country =  df_data.where(df_data.countries_tags.contains(country_tag_name)).select('product_name'
                                                                               ,'categories_tags'
                                                                               ,'nutrition_grade_fr'
                                                                               , 'nutrition-score-fr_100g')
    df_country = df_country.na.drop()
    count_all = df_country.count()
    counted_nutriscore = df_country.groupBy('nutrition_grade_fr').count().sort("nutrition_grade_fr")
    pd_nutriscore = counted_nutriscore.toPandas()
    pd_nutriscore['count'] = (pd_nutriscore['count']/ count_all)*100
    
    #prepare the data to create an interactive histogram
    x=pd_nutriscore['nutrition_grade_fr'].tolist()
    y=pd_nutriscore['count'].tolist()
    country_name = country_tag_name.replace('en:', '').capitalize()
    
    return x,y,country_name

In [15]:
from IPython.display import IFrame

In [16]:
nb_country = 20
liste =(sns.color_palette("hls", nb_country))
data = []
for j in range(0,nb_country):
    country_tag = df_countries['countries_tags'][j]
    labels,values,country_name =NutriScoreCountry(country_tag)
    color_code ='rgb'+ str(liste[j])
    Nutri_score = ['A','B','C','D','E']
    trace = go.Bar(x=Nutri_score,y=values,name=country_name,marker=dict(color=color_code))
    
    data.append(trace)

layout = go.Layout(
    title='Proportion of products by Nutri-Score for different countries',
    xaxis=dict(tickangle=0),
    barmode='group',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='angled-text-bar')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~frederic.bischoff/0 or inside your plot.ly account where it is named 'angled-text-bar'


In [17]:
#to download the html plot
py.plot(fig,filename='Proportion_NutriScore_by_Countries.html')

'https://plot.ly/~frederic.bischoff/10'

The plot above is the first figure on: https://openfactfoodada.github.io/Part1/

According to the graph below, we see that Spain is the country that has the more products of category A avalaible with 23.6 % of the total products and the less products of category E with 12,9%.

At the opposite we see that some countries from Eastern Europe have the fewest products of category A like Czech-republic, Dennmark and Romania 4 %,6,8 % and 9.4  % and are at the same time the countries that have the most products from the baddest category with 51%, 39% and 29% products from category E respectively.

There are also some surprising country like Russia that are in the top 3 of the countries that have the most healthiest products (category A) with 18,3 % but that are also in the top 3 of the countries that have the less healthy products with 39,7% for category E in Russia.

So in next part we will focus on this countries with extrem results but also some other countries with intermediate results that we know well: Switzerland, France and USA.

For these countries we want to see if the difference of distribution of the products in the different catregories (A,B,C,D,E) is due to differents eating habits (there is a specificity in the type product avalaible in these countries) or if there is a difference in the composition of the products (it's not the type of product itself that is specific to the product but the ways it's produced).

For that, in the next part we will first study the distribution of the top tag of products sold in these countries.

## B/ Focus on some countries

In this part we want to see which are the top eight type of products sold in the countries we selected before. The product tag is a feature from OpenFactFood that allows to classify the products. The aim of this part is to see if the differences observed between countries in the first part are due to a specific diversity of product consummed in each of those country (some specific tags in each country due to local habits) or if the tags are the same and in the same distribution, in this case that would mean the difference would be in the composition of the product and not in the type of product.

In [18]:
def eightFirstTagsCountry(country,nb_tags):
    #We create a dataframe with all the data for France

    df_country = df_data.where(df_data.countries_tags.contains(country)).select('product_name'
                                                                               ,'categories_tags'
                                                                               ,'nutrition_grade_fr'
                                                                               , 'nutrition-score-fr_100g')
    df_country = df_country.na.drop()
    
    #We split the categories_tags column to have single category in each row.
    #Then we count the occurrence of each category, and order them from the bigger to the smaller count.

    categories_country = df_country.select(explode(split('categories_tags', ','))).withColumnRenamed("col", "categories_tags")


    pop_cat_country = categories_country.groupBy('categories_tags').count().sort("count", ascending = False)
    country_cat=pop_cat_country.toPandas()
    ten_first=country_cat.head(nb_tags)
    ten_first.loc[:,'count'] = ((ten_first['count'])/(ten_first['count'].sum()))*100
    
    ten_first = ten_first.rename(columns={'count': 'count_'+country})
    
    country_name = country.replace('en:', '').capitalize()

    return (ten_first,country_name)

In [19]:
nb_tags=8
list_countries_tags=['en:france','en:united-states','en:switzerland','en:spain','en:romania','en:russia','en:denmark','en:czech-republic']
list_country=[]
#intialisation with a first tab to which we can add the others
tab,country=eightFirstTagsCountry(list_countries_tags[0],nb_tags)
del list_countries_tags[0]
list_country.append(country)

for countries in list_countries_tags:
    tab1,country=eightFirstTagsCountry(countries,nb_tags)
    list_country.append(country)
    tab=tab.join(tab1.set_index('categories_tags'),how='outer', on='categories_tags')

#clean the names of the categories
tab['categories_tags']=tab['categories_tags'].apply(lambda x: x.replace('en:', '').capitalize())

#fill the nan with zeros
tab=tab.fillna(0)

#reset the index
tab=tab.reset_index(drop=True)

#prepare the data for the bar chart
data=[]
for j in range(0,len(tab)):
    line=tab.loc[j,:].tolist()
    category=line[0]
    del line[0]
    value_category_per_country = line
    
    trace = go.Bar(
    x= list_country,
    y= value_category_per_country,
    name=category
    )
    data.append(trace)

#plot the bar chart   
data = data
layout = go.Layout(
    title='Distribution of the top 8 product-categories for different countries',
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='stacked-bar')

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [20]:
#Save the figure
py.plot(fig,filename='countryTags8.html')

'https://plot.ly/~frederic.bischoff/6'

The plot above is the second figure on: https://openfactfoodada.github.io/Part1/

On the stacked bar chart above, we see that there is an important part of products that are commun to all the selected countries like "plant based foods", "plant based foods and beverages", "sugary snacks", "beverages", but the proportion of these products changes for each countries, we see for example that in Spain where we had particularly good results, the proprortion of plant based foods products is near 30% and the proportion of sugary snack is under 10%, whereas in Czech-Republic the proportion of plant based food is only 13% and the proportion of snacks is over 20%. That explains partially the results obtained in last part.

But we also see that some type of products appears in the top 8 tags in some countries and not in others, for example for Russia,we see that in the top 8 tags we find three types of chocolates "chocolates", "milk chocolate", "dark chocolate" which explains why it was one the country that have the most of "E" products in the first part.

We saw that for each country the representation of the top eight products is not the same and give a first explantation why the proportion of products by categories was so diffrent for some countries but theres is a core of product type that is the same for all the countries, and now we want to go further on this one.

For these categories that are present in all countries: "Plant based food", "Beverages", "Sugary snacks","Dairies","Meals", and "Cereals and potato", we want to know if the distribution of product by categories (A,B,C,D,E) is different or similar. If the distributions are different between the countries, that would be an other element that explains the differences obeserved in first part.


## C/ For given categories and countries

For the categories that are present in all countries: "Plant based food", "Beverages", "Sugary snacks","Dairies", "Meals" and "Cereal and potato", we want to plot the distribution of products by categories (A,B,C,D,E). Here we will focus on Spain the best ranked country in first part, Romania one East Europe country and the three last: Switzerland, USA and France that are commun to us.


In [21]:
liste_labels=['en:plant-based-foods','en:beverages','en:sugary-snacks','en:dairies','en:meals','en:cereals-and-potatoes']

In [22]:
#Function that gives the distribution by category (A,B,C,D,E) for a given tag product and a given country
def prop_cat_country(country_tag,category_tag_name):
    df_country =  df_data.where(df_data.countries_tags.contains(country_tag)).select('product_name'
                                                                               ,'categories_tags'
                                                                               ,'nutrition_grade_fr'
                                                                               , 'nutrition-score-fr_100g').na.drop()

    df_cat= df_country.filter(df_country.categories_tags.contains(category_tag_name))
    tot_country = df_cat.count()

    counted_country = df_cat.groupBy('nutrition_grade_fr').count().sort("nutrition_grade_fr")
    counted_country = counted_country.select(initcap('nutrition_grade_fr'), 'count').withColumnRenamed('initcap(nutrition_grade_fr)', 'nutrition_grade_fr')

    df_counted_country = counted_country.toPandas()
    df_counted_country.loc[:,'count']=(df_counted_country['count'])/(sum(df_counted_country['count']))*100
      
    x=df_counted_country['nutrition_grade_fr'].tolist()
    y=df_counted_country['count'].tolist()
    country_name = country_tag.replace('en:', '').capitalize()
    
    return x,y,country_name

In [23]:
#Function that plot the distribution by category (A,B,C,D,E) for a given tag and a list of countries
def tagsByCategory(category_tag_name):
    list_countries_tags=['en:france','en:united-states','en:switzerland','en:spain','en:romania']
    nb_country = len(list_countries_tags)
    liste =(sns.color_palette("hls", nb_country))
    data=[]
    for j in range (0,nb_country):    
        labels,values,country_name =  prop_cat_country(list_countries_tags[j],category_tag_name)
    
        color_code ='rgb'+ str(liste[j])
        trace = go.Bar(x=labels,y=values,name=country_name,marker=dict(color=color_code))
    
        data.append(trace)
    
        category = category_tag_name.replace('en:', '').capitalize()
    
    layout = go.Layout(
        title='Proportion of ' +category +' products by Nutri-Score for different countries',
        xaxis=dict(tickangle=0),
        barmode='group',
    )

    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig, filename='categories_selected_countries')
    py.plot(fig,filename=category)

In [24]:
#Plot the distribution by category (A,B,C,D,E) for the list of choosen tags and the list of choosen countries
for tag in liste_labels:
    tagsByCategory(tag)

The histograms of this last part are in the thirs part of this page, you can choose for which tag you want to see the histogram: https://openfactfoodada.github.io/Part1/

With the graphs plotted above, we see that for plant based food and sugary snacks there are indeed a greater proportion of "E" products for Romania than in the others countries, that means it's not only the type of product available in a country that makes the distribution of products in categories A,B,C,D,E different but also the quality of these same type products.

First of all, we need to be careful, because the dataset we have is not complete. A lot of products are still not on the OpenFactFood website, and thus are not treated. Furthermore, the product we have are products available in the countries, but we do not know how popular they are, and if people do consume them a lot or not.

# Part 2: What are the differences in the composition of a given product in different countries?

In this part we wanted to compare the amount of sugar contained in different products that looks to be “the same”, across countries. Indeed, we wanted to know if those big companies known all around the world really adapt their products depending on the consumers they want to reach.

In [25]:
df_data.registerTempTable('tab_all')

## Coca-Cola data

In [26]:
#We create the coca dataframe, selecting the Coca-Cola products, with the help of the feature 'product_name'
#We remove the values that seems to be outliers by putting limits for the feature sugars_100g

query = """
SELECT product_name,sugars_100g, countries_tags AS countries
FROM tab_all
WHERE 
(product_name == 'Coca Cola' OR product_name == 'Coca-Cola')
AND 
(sugars_100g>9.5 AND sugars_100g<20)
"""

df_coca = spark.sql(query)
df_coca.show(7)

+------------+-----------+--------------------+
|product_name|sugars_100g|           countries|
+------------+-----------+--------------------+
|   Coca-Cola|         11|    en:united-states|
|   Coca-Cola|       10.8|    en:united-states|
|   Coca-Cola|         11|    en:united-states|
|   Coca-Cola|         11|    en:united-states|
|   Coca Cola|       10.6|           en:france|
|   Coca-Cola|       10.6|en:france,en:swit...|
|   Coca Cola|         10|en:belgium,en:fra...|
+------------+-----------+--------------------+
only showing top 7 rows



In [27]:
#We split the "countries" column to have single country in each row.

df_coca_new = df_coca.select("product_name", "sugars_100g", explode(split('countries', ','))).withColumnRenamed("col", "countries")
df_coca_new.show(5)

+------------+-----------+----------------+
|product_name|sugars_100g|       countries|
+------------+-----------+----------------+
|   Coca-Cola|         11|en:united-states|
|   Coca-Cola|       10.8|en:united-states|
|   Coca-Cola|         11|en:united-states|
|   Coca-Cola|         11|en:united-states|
|   Coca Cola|       10.6|       en:france|
+------------+-----------+----------------+
only showing top 5 rows



In [28]:
#We create a panda dataframe from the spark dataframe

df_coca_pd=df_coca_new.toPandas()
#df_coca_pd

In [29]:
#We check the types of the features we are going to deal with

df_coca_pd.dtypes

product_name    object
sugars_100g     object
countries       object
dtype: object

In [30]:
#We need to convert the type of "sugars_100g" into a numerical type to be able to use the function "aggregate"

df_coca_pd["sugars_100g"] = df_coca_pd["sugars_100g"].astype('float')

In [31]:
#We group by country, and, for each given country, we calculate the median of sugars per 100g
#and order the countries from the bigger to the smaller amount of sugar.

df_coca_g=df_coca_pd.groupby('countries').aggregate(np.median).sort_values("sugars_100g", axis=0, ascending=False)
df_coca_g

,sugars_100g
countries,
en:hungary,11.20
en:slovakia,11.20
en:united-states,11.00
en:czech-republic,10.90
en:georgia,10.90
en:serbia,10.75
en:south-africa,10.60
en:spain,10.60
en:sweden,10.60


In [32]:
#min value
df_coca_g.sugars_100g.min()

10.4

In [33]:
#max value
df_coca_g.sugars_100g.max()

11.2

With the median of the amount of sugar we see that for the United States the amount of sugar is approximately 0.4 g (for 100g of coca) higher than in european countries.

## Haribo Gold Bears data

In [34]:
#We create the Haribo dataframe, using the feature 'brands'

query = """
SELECT product_name,sugars_100g, brands, countries_tags AS countries
FROM tab_all
WHERE 
(brands RLIKE 'Haribo.*')
OR
(brands RLIKE 'haribo.*')
"""

df_haribo = spark.sql(query)
df_haribo.show(5, False)

+-----------------------+-----------+------------------------------------------------+----------------+
|product_name           |sugars_100g|brands                                          |countries       |
+-----------------------+-----------+------------------------------------------------+----------------+
|Happy Cola Candy       |55.5       |Haribo,  Haribo Gmbh & Co. Kg                   |en:united-states|
|Gold Bears Gummi Candy |50         |Haribo,  Haribo Of America  Inc.                |en:united-states|
|Gold-Bears Gummi Candy |53.85      |Haribo                                          |en:united-states|
|Gold-Bears, Gummi Candy|46.15      |Haribo,  Haribo Sekerleme San. Ve Tic. Ltd. Sti.|en:united-states|
|Gold-Bears, Gummi Candy|46.15      |Haribo                                          |en:united-states|
+-----------------------+-----------+------------------------------------------------+----------------+
only showing top 5 rows



In [35]:
#We split the "countries" column to have single country in each row.

df_haribo_new = df_haribo.select("product_name", "sugars_100g", "brands", explode(split('countries', ','))).withColumnRenamed("col", "countries")
df_haribo_new.show(5)

+--------------------+-----------+--------------------+----------------+
|        product_name|sugars_100g|              brands|       countries|
+--------------------+-----------+--------------------+----------------+
|    Happy Cola Candy|       55.5|Haribo,  Haribo G...|en:united-states|
|Gold Bears Gummi ...|         50|Haribo,  Haribo O...|en:united-states|
|Gold-Bears Gummi ...|      53.85|              Haribo|en:united-states|
|Gold-Bears, Gummi...|      46.15|Haribo,  Haribo S...|en:united-states|
|Gold-Bears, Gummi...|      46.15|              Haribo|en:united-states|
+--------------------+-----------+--------------------+----------------+
only showing top 5 rows



In [36]:
df_haribo_new.registerTempTable('tab_haribo')

We decided to focus on the "Gold Bears" type of Haribo Candy, but to be sure that we are talking about the "same" product across countries.

In [37]:
#We create the Gold Bears dataframe, from the Haribo dataframe

query = """
SELECT product_name,sugars_100g, brands, countries
FROM tab_haribo
WHERE 
((product_name RLIKE 'Bears.*')
OR
(product_name RLIKE 'Ours.*')
OR
(product_name RLIKE 'Gold.*')
OR
(product_name RLIKE 'Oro.*')
OR
(product_name RLIKE 'Aur.*')
OR
(product_name RLIKE 'Aur.*')
OR
(product_name RLIKE 'Urs.*'))
AND
(product_name NOT RLIKE ',')
"""
#Last line because we don't want sweet mixes, we only want the gold bears alone

df_gold_bears = spark.sql(query)
df_gold_bears.show(10, False)

#df_haribo.count()

+---------------------------+-----------+--------------------------------+----------------+
|product_name               |sugars_100g|brands                          |countries       |
+---------------------------+-----------+--------------------------------+----------------+
|Gold Bears Gummi Candy     |50         |Haribo,  Haribo Of America  Inc.|en:united-states|
|Gold-Bears Gummi Candy     |53.85      |Haribo                          |en:united-states|
|Gold-Bears Gummi Candy     |53.85      |Haribo                          |en:united-states|
|Gold-Bears Gummi Candy     |48.72      |Haribo,  Haribo Of America  Inc.|en:united-states|
|Sour Gold-Bears Gummi Candy|53.66      |Haribo,  Haribo Of America  Inc.|en:united-states|
|Gold-Bears Gummi Candy     |53.66      |Haribo Of America  Inc.         |en:united-states|
|Sour Gold-Bears            |53.66      |Haribo,  Haribo Of America  Inc.|en:united-states|
|Gold Bears Gummi Candy     |48.72      |Haribo Of America  Inc.         |en:uni

In [38]:
#We create a panda dataframe from the spark dataframe

df_gold_bears_pd=df_gold_bears.toPandas()

#Then, we need to convert the type of "sugars_100g" into a numerical type to be able to use the function "aggregate"

df_gold_bears_pd["sugars_100g"] = df_gold_bears_pd["sugars_100g"].astype('float')

#We group by country, and, for each given country, we calculate the median of sugars per 100g
#and order the countries from the bigger to the smaller amount of sugar.

df_gold_bears_g=df_gold_bears_pd.groupby('countries').aggregate(np.median).sort_values("sugars_100g", axis=0, ascending=False)
df_gold_bears_g

,sugars_100g
countries,
en:united-kingdom,58.40
en:united-states,53.66
en:austria,46.00
en:china,46.00
en:france,46.00
en:germany,46.00
en:ireland,46.00
en:italy,46.00
en:switzerland,46.00


In [39]:
#min value
df_gold_bears_g.sugars_100g.min()

46.0

In [40]:
#max value
df_gold_bears_g.sugars_100g.max()

58.4

## Nutella data

In [41]:
#We create the nutella dataframe

query = """
SELECT product_name,sugars_100g,countries_tags AS countries
FROM tab_all
WHERE 
(product_name == 'Nutella')
AND 
(sugars_100g>20 AND sugars_100g<80)
"""
#We do not keep really aberrant values for sugars_100

df_nutella = spark.sql(query)
df_nutella.show(5)

+------------+-----------+--------------------+
|product_name|sugars_100g|           countries|
+------------+-----------+--------------------+
|     Nutella|       56.8|en:canada,en:swit...|
|     Nutella|       56.8|    en:united-states|
|     Nutella|       56.8|    en:united-states|
|     Nutella|       57.6|        en:australia|
|     Nutella|         58|en:canada,en:fran...|
+------------+-----------+--------------------+
only showing top 5 rows



In [42]:
#We split the "countries" column to have single country in each row.

df_nutella_new = df_nutella.select("product_name", "sugars_100g", explode(split('countries', ','))).withColumnRenamed("col", "countries")

#We create a panda dataframe from the spark dataframe

df_nutella_pd=df_nutella_new.toPandas()

#Then, we need to convert the type of "sugars_100g" into a numerical type to be able to use the function "aggregate"

df_nutella_pd["sugars_100g"] = df_nutella_pd["sugars_100g"].astype('float')

#We group by country, and, for each given country, we calculate the median of sugars per 100g
#and order the countries from the bigger to the smaller amount of sugar.

df_nutella_g=df_nutella_pd.groupby('countries').aggregate(np.median).sort_values("sugars_100g", axis=0, ascending=False)

#We delete the Quebec, that is not a country and thus probably a mistake
df_nutella_g = df_nutella_g.drop(df_nutella_g.index[0])

df_nutella_g.head(8)

,sugars_100g
countries,
en:australia,57.6
en:canada,56.9
en:hungary,56.8
en:united-states,56.8
en:martinique,56.8
en:mali,56.8
en:algeria,56.8
en:switzerland,56.7


In [43]:
#min value
df_nutella_g.sugars_100g.min()

55.95

In [44]:
#max value
df_nutella_g.sugars_100g.max()

57.6

## Creating the map for each of the 3 products

In [45]:
#We set index on the dataframes

df_coca_g = df_coca_g.reset_index()
df_gold_bears_g = df_gold_bears_g.reset_index()
df_nutella_g = df_nutella_g.reset_index()

In [46]:
#We create only one dataframe containing all the country names present for the 3 different products

list_country_1 = df_coca_g.iloc[:,0]
list_country_2 = df_gold_bears_g.iloc[:,0]
list_country_3 = df_nutella_g.iloc[:,0]

frames = [list_country_1, list_country_2, list_country_3]

df_list_countries = pd.concat(frames)
df_list_countries = df_list_countries.drop_duplicates()
df_list_countries.head(5)

0           en:hungary
1          en:slovakia
2     en:united-states
3    en:czech-republic
4           en:georgia
Name: countries, dtype: object

In [47]:
#We reset the index of this new country names dataframe

df_list_countries = df_list_countries.sort_values().reset_index(drop=True)
df_list_countries.head(8)

0      en:algeria
1    en:argentina
2    en:australia
3      en:austria
4      en:belgium
5       en:brazil
6     en:cameroon
7       en:canada
Name: countries, dtype: object

In [48]:
#We create a list containing the ISO_3 codes for the list of countries from the previous dataframe

list_ISO_3 = ['DZA', 'ARG', 'AUS', 'AUT', 'BEL', 'BRA', 'CMR', 'CAN', 'CHN', 'COL', 'CZE', 'DNK', 'FRA', 'GEO', 'DEU', 'GRC', 'GLP', 'HUN', 'IND','IRQ', 'IRL',
              'ITA', 'JPN', 'LUX', 'MLI', 'MTQ', 'MEX', 'MAR', 'NLD', 'NCL', 'NGA', 'PAK', 'PHL', 'PRT', 'REU', 'RUS', 'SAU', 'SEN', 'SRB', 'SVK', 'SVN', 'ZAF',
              'ESP', 'SWE', 'CHE', 'TWN', 'TUN', 'GBR', 'USA']

In [49]:
#We create this conversion dataframe

country_to_ISO = pd.DataFrame({'countries': df_list_countries, 'ISO': list_ISO_3})
country_to_ISO.head(8)

,countries,ISO
0,en:algeria,DZA
1,en:argentina,ARG
2,en:australia,AUS
3,en:austria,AUT
4,en:belgium,BEL
5,en:brazil,BRA
6,en:cameroon,CMR
7,en:canada,CAN


In [50]:
#COCA

#We merge the two dataframe, so we have all our coca datas linked to the ISO code
coca_ISO = pd.merge(df_coca_g, country_to_ISO, on = 'countries')

#we remove the Martinique row (index 34)
coca_ISO = coca_ISO.drop(coca_ISO.index[34])
#And the Reunion row (index 14)
coca_ISO = coca_ISO.drop(coca_ISO.index[14])
#Because those 2 are not 'countries' and thus are not present on our geo_json file

#Then we rest the index
coca_ISO = coca_ISO.sort_values('ISO').reset_index()
coca_ISO.head(6)

,index,countries,sugars_100g,ISO
0,20,en:australia,10.6,AUS
1,33,en:austria,10.6,AUT
2,32,en:belgium,10.6,BEL
3,31,en:brazil,10.6,BRA
4,29,en:canada,10.6,CAN
5,10,en:switzerland,10.6,CHE


In [51]:
#NUTELLA

#We merge the two dataframe, so we have all our Nutella datas linked to the ISO code
nutella_ISO = pd.merge(df_nutella_g, country_to_ISO, on = 'countries')

#we remove the Guadeloupe row (index 30)
nutella_ISO = nutella_ISO.drop(nutella_ISO.index[30])
#and the Martinique row (index 4)
nutella_ISO = nutella_ISO.drop(nutella_ISO.index[4])
#Because those 2 are not 'countries' and thus are not present on our geo_json file

#Then we rest the index
nutella_ISO = nutella_ISO.sort_values('ISO').reset_index()
nutella_ISO.head(6)

,index,countries,sugars_100g,ISO
0,26,en:argentina,56.3,ARG
1,0,en:australia,57.6,AUS
2,12,en:austria,56.3,AUT
3,9,en:belgium,56.7,BEL
4,1,en:canada,56.9,CAN
5,7,en:switzerland,56.7,CHE


In [52]:
#GOLD BEARS

#We merge the two dataframe, so we have all our Gold Bears datas linked to the ISO code
gold_bear_ISO = pd.merge(df_gold_bears_g, country_to_ISO, on = 'countries')

#We drop the NaN rows for sugars_100g
gold_bear_ISO = gold_bear_ISO.dropna()

gold_bear_ISO = gold_bear_ISO.sort_values('ISO').reset_index()
gold_bear_ISO

,index,countries,sugars_100g,ISO
0,2,en:austria,46.00,AUT
1,8,en:switzerland,46.00,CHE
2,3,en:china,46.00,CHN
3,5,en:germany,46.00,DEU
4,4,en:france,46.00,FRA
5,0,en:united-kingdom,58.40,GBR
6,6,en:ireland,46.00,IRL
7,7,en:italy,46.00,ITA
8,1,en:united-states,53.66,USA


In [53]:
#We prepare the dataframes that will be used for the map

coca_map = coca_ISO.set_index('ISO')['sugars_100g']
df_coca_map = pd.DataFrame(coca_map)

nutella_map = nutella_ISO.set_index('ISO')['sugars_100g']
df_nutella_map = pd.DataFrame(nutella_map)

gold_bear_map = gold_bear_ISO.set_index('ISO')['sugars_100g']
df_gold_bear_map = pd.DataFrame(gold_bear_map)

In [54]:
#We define the path to the 3 geo_json files

countries_coca_geo_path = './coca.geo.json'
geo_coca_data = json.load(open(countries_coca_geo_path))

countries_nutella_geo_path = './nutella.geo.json'
geo_nutella_data = json.load(open(countries_nutella_geo_path))

countries_gold_bear_geo_path = './gold_bears.geo.json'
geo_gold_bear_data = json.load(open(countries_gold_bear_geo_path))

In [55]:
#Add the value of the country into the json file. It allows to show the real value of the country when passing
#on the country in the map

def augment_topo_json(json, dataframe, transform=(lambda col, val: val)):
    for countries in json['features']:
        for column in dataframe.columns.values:
            countries['properties'][column] = transform(column, dataframe.loc[countries['id']][column])

In [56]:
#We apply the previously defined function on our 3 geo_json files

augment_topo_json(geo_coca_data, df_coca_map, \
                  (lambda col, val: 'No Data' if np.isnan(val) else f'{val:.2f}'))

augment_topo_json(geo_nutella_data, df_nutella_map, \
                  (lambda col, val: 'No Data' if np.isnan(val) else f'{val:.2f}'))

augment_topo_json(geo_gold_bear_data, df_gold_bear_map, \
                  (lambda col, val: 'No Data' if np.isnan(val) else f'{val:.2f}'))

## Coca map

In [57]:
#We define the color function we want to use: here 10 step of Pink color for our range of values (10.3 to 11.3)
linear = cm.linear.PuRd_07.to_step(10).scale(10.3,11.3)

def my_color_function_coca(feature):
    return linear(coca_map[feature['id']])

map_coca_world = folium.Map([45,5], zoom_start =2)

def style_function(feature):
    return {
        'fillOpacity': 0.6,
        'color': 'black',
        'weight': 0.7,
        'fillColor': my_color_function_coca(feature)
    }


folium.GeoJson(geo_coca_data, style_function=style_function,
               tooltip=folium.features.GeoJsonTooltip(['name', 'sugars_100g'],
        aliases=['Country :', 'value :'])).add_to(map_coca_world)

linear.caption = 'Percentage of sugar in Coca-Cola (grams of sugars in 100g of Coca-cola)'

map_coca_world.add_child(linear)

#We save it into a html file to be able to put it on our website
map_coca_world.save('Coca_map.html')
map_coca_world

First map of the second page of the website: https://openfactfoodada.github.io/Part2/

The overall tendency on this first map is a bigger amount of sugar in the United states and in the Eastern countries, compared to the rest of the world.

## Nutella map

In [58]:
#We define the color function we want to use: here 18 step of Pink color for our range of values (55.9 to 57.7)
linear = cm.linear.PuRd_07.to_step(18).scale(55.9,57.7)

def my_color_function_nutella(feature):
    return linear(nutella_map[feature['id']])

map_nutella_world = folium.Map([45,5], zoom_start =2)

def style_function(feature):
    return {
        'fillOpacity': 0.6,
        'color': 'black',
        'weight': 0.7,
        'fillColor': my_color_function_nutella(feature)
    }


folium.GeoJson(geo_nutella_data, style_function=style_function,
              tooltip=folium.features.GeoJsonTooltip(['name', 'sugars_100g'],
        aliases=['Country :', 'value :'])).add_to(map_nutella_world)

linear.caption = 'Percentage of sugar in Nutella (grams of sugars in 100g of Nutella)'

map_nutella_world.add_child(linear)

#We save it into a html file to be able to put it on our website
map_nutella_world.save('Nutella_map.html')
map_nutella_world

Second map of the second page of the website: https://openfactfoodada.github.io/Part2/

This second map shows greater amount of sugar in the United States, Canada, Australia, Hungary, as well as Algeria and Mali, compared to the rest of the world.

## Gold Bears map

In [59]:
#We define the color function we want to use: here 10 step of Pink color for our range of values (40 to 60)
linear = cm.linear.PuRd_07.to_step(10).scale(40,60)

def my_color_function_gold_bear(feature):
    return linear(gold_bear_map[feature['id']])

map_gold_bear_world = folium.Map([45,5], zoom_start =2)

def style_function(feature):
    return {
        'fillOpacity': 0.6,
        'color': 'black',
        'weight': 0.7,
        'fillColor': my_color_function_gold_bear(feature)
    }


folium.GeoJson(geo_gold_bear_data, style_function=style_function,
              tooltip=folium.features.GeoJsonTooltip(['name', 'sugars_100g'],
        aliases=['Country :', 'value :'])).add_to(map_gold_bear_world)

linear.caption = 'Percentage of sugar in Gold Bears (Haribo sweets) (grams of sugars in 100g)'

map_gold_bear_world.add_child(linear)

#We save it into a html file to be able to put it on our website
map_gold_bear_world.save('Gold_bear_map.html')
map_gold_bear_world

Third map of the second page of the website: https://openfactfoodada.github.io/Part2/

This last map shows only difference for the United States and the United Kingdom, that present greater amount of sugar in comparison to the rest of the countries).

# Part 3: How much of the food that we buy ar local
### Which countries consume more locally?

The map (with pie chart) of this part can be found on the third page of our website: https://openfactfoodada.github.io/part3/

One important data field is called manufacturing_places and represents the place where the food product is manufactured. Another field is the origins which represent the origin of the food present in the product.

In [60]:
#Selecting the more important columns
dfbis=df_data.select("countries", "product_name","categories","manufacturing_places","origins")
dfbis.show(5)

+---------+--------------------+----------+--------------------+-------+
|countries|        product_name|categories|manufacturing_places|origins|
+---------+--------------------+----------+--------------------+-------+
|en:france|    Vitória crackers|      null|                null|   null|
|    en:FR|               Cacao|      null|                null|   null|
|en:france|Sauce Sweety chil...|      null|                null|   null|
|en:france|           Mendiants|      null|                null|   null|
|en:france|Salade de carotte...|      null|                null|   null|
+---------+--------------------+----------+--------------------+-------+
only showing top 5 rows



## 1 Distance traveling of food products

The aim of this part is to calculate the distance between the manufacturing places or the origins of the food products and the country where they are sold. For each countries a mean is calculated.

Drawback : -Concerning the country where the product is sold only one coordinate value is given, around the centre of the country which usually not correspond to the real coordinate where the product is sold.
The same apply if only the country is given for the manufacturing place or the origin of the products creating a bias around the real number of kilometers that the food product has traveled.

### Data cleaning

A lot of country names begin with en:, it is necessary to supress it. Furthermore, one row can contains several country names.

In [67]:
#Clean the name of countries and create a list of countries if multiple countries are given
def cleanCountries(countries):
    if countries is not None :
        countries=countries.replace('en:', '').split(',')
    return countries
    
clean_udf = udf(lambda x : cleanCountries(x), ArrayType(StringType()))

In [68]:
#Create a DataFrame with the name of the countries cleaned
df_clean = dfbis.select(clean_udf('countries').alias('countries'),
                        'product_name',
                        'categories',
                        'manufacturing_places',
                        'origins',
                       )

In [69]:
#Explode the list of countries into one country per row
df_clean_final = df_clean.withColumn("countries", explode(df_clean.countries))

In [70]:
df_clean_final.show(5)

+---------+--------------------+----------+--------------------+-------+
|countries|        product_name|categories|manufacturing_places|origins|
+---------+--------------------+----------+--------------------+-------+
|   france|    Vitória crackers|      null|                null|   null|
|       FR|               Cacao|      null|                null|   null|
|   france|Sauce Sweety chil...|      null|                null|   null|
|   france|           Mendiants|      null|                null|   null|
|   france|Salade de carotte...|      null|                null|   null|
+---------+--------------------+----------+--------------------+-------+
only showing top 5 rows



The names of the countries are written in different languages either in uppercase or lowercase. One way to get rid of this difficulty is to use the utility of python called babel. With the isocode of languages and countries, one list of names of countries can be made in different languages.

In [71]:
#Create a list of country names in different languages with respect to lower and upper case
#The country is given with respect to the norm ISO 3166 alpha 2
#The language is given with respect to the norm ISO 639 alpha 2 
def iso_3166_1_list(isocode) :
    list_names = []
    for language in ('en', 'fr', 'de', 'es', 'ar', 'zh', 'ru', 'it', 'nl', 'sv', 'pt', 'da', 'hu', 'ro'):
        locale = Locale(language)
        list_names.append(locale.territories[isocode])
        list_names.append(locale.territories[isocode].lower())
    list_names.append(isocode)
    list_names.append(isocode.lower())
    if isocode == 'US' :
        list_names.append('USA')
        list_names.append('USA'.lower()) 
        list_names.append('United states')
    if isocode == 'GB' :
        list_names.append('United kingdom')
    if isocode == 'MX' :
        list_names.append('MEXICO')
    if isocode == 'AE' :
        list_names.append('EUA')
        list_names.append('eua')
    if isocode == 'EU' :
        list_names.append('Agricultura UE,Agricultura no UE')
        list_names.append('Agricultura UE')
    return list_names

Some origins and manufacturing places containes Neprecizat or Neprecizata which mean unspecified. We decide to handle these names as no origins or manufacturing places given.

In [72]:
#Check if the string given contain one of the names present in the list
#If it is the case give the generic country name
def is_country(country_name, list_countries):
    if country_name == 'Neprecizat' or country_name == 'Neprecizata':
        country_name = None
    if country_name is not None :
        if any(x in country_name for x in list_countries) == True : 
            country_name = list_countries[28]
    return country_name

def country_udf(countries_list) :
    return udf(lambda x : is_country(x, countries_list), StringType())

We interested only with the 20 countries with the more informations given. Indeed, with more than 20 countries the computation time would be too long and the others countries have very few informations given to give an ideal representation of the effect happening in the country (not enough food products to have good statistical power).
We clean the data with China, Thailand, New Zealand, Emirates Arab Union, Serbia, European Union, Vietnam an Sri Lanka as well because these countries are often present as manufacturing places or origins of the food products.

In [76]:
for countries in (['FR', 'US', 'DE', 'CH', 'BE', 'ES', 'GB', 'CA', 'IT', 'IE']):
    df_clean_final = df_clean_final.select(country_udf(iso_3166_1_list(countries))(col('countries')).alias('countries'),
                         'product_name',
                         'categories', 
                        'manufacturing_places',
                        'origins')

In [77]:
for countries in (['MX', 'AU', 'RU', 'NL', 'SE', 'PT', 'LU', 'DK', 'RO', 'HU']):
    df_clean_final = df_clean_final.select(country_udf(iso_3166_1_list(countries))(col('countries')).alias('countries'),
                         'product_name',
                         'categories', 
                        'manufacturing_places',
                        'origins')

In [78]:
for countries in (['CN', 'TH', 'NZ', 'AE', 'RS', 'EU', 'VN', 'LK']):
    df_clean_final = df_clean_final.select(country_udf(iso_3166_1_list(countries))(col('countries')).alias('countries'),
                         'product_name',
                         'categories', 
                        'manufacturing_places',
                        'origins')

In [79]:
df_clean_final.show(5)

+---------+--------------------+----------+--------------------+-------+
|countries|        product_name|categories|manufacturing_places|origins|
+---------+--------------------+----------+--------------------+-------+
|       FR|    Vitória crackers|      null|                null|   null|
|       FR|               Cacao|      null|                null|   null|
|       FR|Sauce Sweety chil...|      null|                null|   null|
|       FR|           Mendiants|      null|                null|   null|
|       FR|Salade de carotte...|      null|                null|   null|
+---------+--------------------+----------+--------------------+-------+
only showing top 5 rows



### Distance calculation using the geodesic distance

One way of computing the distance that the food product has traveled is to generate the latitude and longitude of the location given using geopy. The distance between two coordinates is then calculated with the geodesic distance as a default from geopy. However, Geopy has a limited number of requests. One way to avoid this issue is to take the 20 most used adress for the origin or manufacturing place of the country of interest and calculate the distance from these adresses to the centre of the country. A mean of the distance weighted by the number of occurence of the adress is then calculated. Geopy is very efficient to calculate longitude and latitude of an adress written in multiple languanges and multiple forms.

In [66]:
#Calculate the latitude and longitude of a specified adress
def coordinates_estimator(places):
    geocoder = Nominatim()
    location = geocoder.geocode(places, 4)
    if location is not None :
        return [location.latitude, location.longitude]
    else : 
        return None

In [ ]:
#Calculate the geodesic distance given the latitude and longitude of two points
def distance_calculator(countries_coor, places_coor) :
    if places_coor is not None :
        return distance.distance(countries_coor, places_coor).km
    else : 
        return None

In [ ]:
#Give a kilometer average that the product has to travel before being sold in the country of interest
#Specify the manufacturing_places or origins of the product
def distance_focus(country_name, header):
    list_header = []
    total_km = 0
    total_count = 0
    result = df_clean_final[df_clean_final['countries'] == country_name]
    df_distance = result.groupby(header).count().toPandas()
    df_distance = df_distance.dropna().sort_values(by = ['count'], ascending = False).head(20)
    df_distance[header] = df_distance[header].apply(lambda x : coordinates_estimator(x))
    country_coor = coordinates_estimator(country_name)
    df_distance[header] = df_distance[header].apply(lambda x : distance_calculator(country_coor, x))
    df_distance = df_distance.dropna().reset_index()
    for i in range(len(df_distance)):
        total_km = total_km + df_distance[header][i]*df_distance['count'][i]
        total_count = total_count + df_distance['count'][i]
    avg_km = total_km/total_count
    return avg_km

The dataframe containing all the mean distances with respect of the manufacturing places and origins is called df_distance_global

In [ ]:
df_distance_global = pd.DataFrame(index = np.arange(40), columns = ['countries',
                                 'headers',
                                 'average number of km'])
counter = 0
for countries in (['FR', 'US', 'DE', 'CH', 'BE', 'ES', 'GB', 'CA', 'IT', 'IE']):
    for headers in (['manufacturing_places', 'origins']):
        df_distance_global['countries'][counter] = countries
        df_distance_global['headers'][counter] = headers
        df_distance_global['average number of km'][counter] = distance_focus(countries, headers)
        counter = counter + 1

In [ ]:
for countries in (['MX', 'AU', 'RU', 'NL', 'SE', 'PT', 'LU', 'DK', 'RO', 'HU']):
    for headers in (['manufacturing_places', 'origins']):
        df_distance_global['countries'][counter] = countries
        df_distance_global['headers'][counter] = headers
        df_distance_global['average number of km'][counter] = distance_focus(countries, headers)
        counter = counter + 1

In [ ]:
df_distance_global.to_csv('df_distance.csv', sep = '\t')

## 2 Countries characterization

After seeing how many kilometers a product has traveled, we can now focus from which countries the products originated from. This allow to see if the country import a lot of their food.

Drawback : -Only manufacturing places or origins where one name of the country is given are taken into account. For instance if only Strasbourg is given, it will not be registered as part of the France.

### Data cleaning

The data from the manufacturing_places and the origins has to be cleaned in order to extract the names of the countries from the adress given. We take the 20 same countries in order to have a coherence between the two parts.

In [80]:
#Clean the data using the list of countries given
for countries in (['FR', 'US', 'DE', 'CH', 'BE', 'ES', 'GB', 'CA', 'IT', 'IE']):
    df_clean_final = df_clean_final.select ('countries',
                         'product_name',
                         'categories', 
                         country_udf(iso_3166_1_list(countries))(col('manufacturing_places')).alias('manufacturing_places'),
                         country_udf(iso_3166_1_list(countries))(col('origins')).alias('origins'))

In [81]:
#Clean the data using the list of countries given
for countries in (['MX', 'AU', 'RU', 'NL', 'SE', 'PT', 'LU', 'DK', 'RO', 'HU']):
    df_clean_final = df_clean_final.select(country_udf(iso_3166_1_list(countries))(col('countries')).alias('countries'),
                         'product_name',
                         'categories', 
                         country_udf(iso_3166_1_list(countries))(col('manufacturing_places')).alias('manufacturing_places'),
                         country_udf(iso_3166_1_list(countries))(col('origins')).alias('origins'))

In [82]:
#Clean the data using the list of countries given
for countries in (['CN', 'TH', 'NZ', 'AE', 'RS', 'EU', 'VN', 'LK']):
    df_clean_final = df_clean_final.select(country_udf(iso_3166_1_list(countries))(col('countries')).alias('countries'),
                         'product_name',
                         'categories', 
                         country_udf(iso_3166_1_list(countries))(col('manufacturing_places')).alias('manufacturing_places'),
                         country_udf(iso_3166_1_list(countries))(col('origins')).alias('origins'))

In [83]:
df_clean_final.show(5)

+---------+--------------------+----------+--------------------+-------+
|countries|        product_name|categories|manufacturing_places|origins|
+---------+--------------------+----------+--------------------+-------+
|       FR|    Vitória crackers|      null|                null|   null|
|       FR|               Cacao|      null|                null|   null|
|       FR|Sauce Sweety chil...|      null|                null|   null|
|       FR|           Mendiants|      null|                null|   null|
|       FR|Salade de carotte...|      null|                null|   null|
+---------+--------------------+----------+--------------------+-------+
only showing top 5 rows



### Percentage of local country

A percentage is calculated representing the proportion of food products that is originated or manufactured in the same country where the product is sold.

In [64]:
#Change the isocode of the country to the english name of the country
def change_name(isocode):
    return iso_3166_1_list(isocode)[0]

In [63]:
#Focus on a particular country in term of origin of products or manufacturing places
#The country_name represents the name of the country that we focus on
#The headers can be either origins or manufacturing_places 
def country_focus(country_name, headers) :
    result = df_clean_final[df_clean_final['countries'] == country_name]
    df_local_plus = result.groupby(headers).count().toPandas()
    df_local_plus = df_local_plus.dropna()
    others = df_local_plus['count'].sum()
    df_local = df_local_plus.sort_values(by = ['count'], ascending = False).head(5)
    df_local[headers] = df_local[headers].apply(lambda x : change_name(x))
    others2 = others - df_local['count'].sum()
    dfappend = pd.DataFrame({headers : ['Others countries'],
                                   'count' : [others2]}, index = [len(df_local_plus)+1])
    df_local = df_local.append(dfappend).reset_index()
    return df_local, others

In [65]:
#Calculate the percentage of food products that originated or are manufactured in the own country
def percent_calculator(country_name, headers):
    df_loc, others = country_focus(country_name, headers)
    if df_loc[df_loc[headers] == change_name(country_name)].empty == True :
        return 0
    else : 
        return (df_loc[df_loc[headers] == change_name(country_name)]['count'].values[0]/others)*100

The DataFrame where the proportion are stored for the 20 countries either as manufacturing places or origins is called df_countries_global

In [ ]:
df_countries_global = pd.DataFrame(index = np.arange(40), columns = ['countries',
                                 'headers',
                                 'percent from the same country'])
counter = 0
for countries in (['FR', 'US', 'DE', 'CH', 'BE', 'ES', 'GB', 'CA', 'IT', 'IE']):
    for headers in (['manufacturing_places', 'origins']):
        df_countries_global['countries'][counter] = countries
        df_countries_global['headers'][counter] = headers
        df_countries_global['percent from the same country'][counter] = percent_calculator(countries, headers)
        counter = counter + 1

In [ ]:
for countries in (['MX', 'AU', 'RU', 'NL', 'SE', 'PT', 'LU', 'DK', 'RO', 'HU']):
    for headers in (['manufacturing_places', 'origins']):
        df_countries_global['countries'][counter] = countries
        df_countries_global['headers'][counter] = headers
        df_countries_global['percent from the same country'][counter] = percent_calculator(countries, headers)
        counter = counter + 1

In [ ]:
df_countries_global.to_csv('df_countries.csv', sep = '\t')

### Creating PNG pie charts for the map

A popup containing a pie chart.png is added to the map corresponding to the percent of local products. The pie chart is plotted with the percent of food products manufactured or originated from the 5 most biggest countries and from the others countries.

In [62]:
#Focus on a particular country in term of origin of products or manufacturing places
#The country_name represents the name of the country that we focus on
#The headers can be either origins or manufacturing_places 
def country_focus_png(country_name, headers) :
    result = df_clean_final[df_clean_final['countries'] == country_name]
    df_local_plus = result.groupby(headers).count().toPandas()
    df_local_plus = df_local_plus.dropna()
    others = df_local_plus['count'].sum()
    df_local = df_local_plus.sort_values(by = ['count'], ascending = False).head(5)
    df_local[headers] = df_local[headers].apply(lambda x : change_name(x))
    others2 = others - df_local['count'].sum()
    dfappend = pd.DataFrame({headers : ['Others countries'],
                                   'count' : [others2]}, index = [len(df_local_plus)+1])
    df_local = df_local.append(dfappend).reset_index()
    index = df_local.index[df_local[headers] == change_name(country_name)].tolist()
    explode = [0, 0, 0, 0, 0, 0]
    explode[index[0]] = 0.1
    fig, ax = plt.subplots(figsize=(7, 4))
    ax.pie(df_local['count'], labels = df_local[headers], explode = explode, autopct='%1.1f%%',shadow=True, startangle=90)
    fig.suptitle('Proportion of food products by ' + headers + ' in ' + change_name(country_name))
    resolution, width, height = 75, 7, 4
    station = '42'
    png = country_name + '-' + headers + '.png'.format(station)
    fig.savefig(png, dpi=resolution)
    encoded = base64.b64encode(open(png, 'rb').read()).decode()
    html ='<img src="data:image/png;base64,{}">'.format
    iframe = folium.IFrame(html(encoded), width=(width*resolution)+20, height=(height*resolution)+20)
    popup = folium.Popup(iframe, max_width=2650)
    return popup

## 3 Mapping of the results

A map of all the results is implemented with the 20 countries that we focus on. Each country has a color that depend on the value. For the local part, a country that appear green mean that this country have their that come from their own whereas red reflect the opposite.

In [84]:
#Make a list of countries that can be easily showed in a map
#Choose one of the two DataFrames (df_distance_global or df_countries_global)
#Choose between manufacturing_places and origins
#Choose the color for the scale of countries
#Choose if the scale begin and end with a round number or not
#Return the interesting data and the color linear scale
def make_list_map(dataframe, header_choose, color_scale, round_linear = False):
    dataframe = dataframe[dataframe['headers'] == header_choose]
    dataframe = dataframe.drop(['headers'], axis=1)
    final = dataframe.set_index('countries')[dataframe.columns.values[-1:][0]]
    if round_linear == True:
        maxi = round(max(final), -2)
        mini = round(min(final), -2)
    else: 
        maxi = max(final)
        mini = min(final)
    #Create a linear color bar
    linear1 = color_scale.scale(mini,maxi)
    return final, linear1

In [98]:
list_map, linear = make_list_map(df_distance_global, 'manufacturing_places', cm.linear.PuOr_04.to_step(10), round_linear = True)
df_list_map = pd.DataFrame(list_map)

world.geo.json is a json file containing the 20 countries that we focus on

In [99]:
world_path = 'world.geo.json'
geo_json_data = json.load(open(world_path))

In [100]:
#Add the value of the country into the json file. It allows to show the real value of the country when passing
#on the country in the map
def augment_topo_json(json, dataframe, transform=(lambda col, val: val)):
    for countries in json['features']:
        for column in dataframe.columns.values:
            countries['properties'][column] = transform(column, dataframe.loc[countries['id']][column])

In [101]:
augment_topo_json(geo_json_data, df_list_map, \
                  (lambda col, val: 'No Data' if np.isnan(val) else f'{val:.2f}'))

In [102]:
#Create a function that give a color in function of the value
def my_color_function1(country_name, feature):
    value = df_list_map.loc[df_list_map.index == country_name].values[0][0]
    return linear(value)

In [103]:
#Create the style of the map like the border of each countries and the color of the countries
def style_function1(feature):
    return {
        'fillOpacity': 0.6,
        'color': 'black',
        'weight': 1,
        'fillColor': my_color_function1(feature['id'], feature)
    }

In [106]:
m = folium.Map([45, 0], zoom_start = 2)
folium.GeoJson(geo_json_data,
                style_function=style_function1,
              tooltip=folium.features.GeoJsonTooltip(['name', 'average number of km'],
        aliases=['Country :', 'value :'])).add_to(m)
#for countries in (['FR', 'US', 'DE', 'CH', 'BE', 'ES', 'GB', 'CA', 'IT', 'IE']):
    #popup = country_focus_png(countries, 'manufacturing_places')
    #marker = folium.Marker(coordinates_estimator(countries), popup = popup)
    #m.add_child(marker)
m.add_child(linear)
linear.caption = "Average number of km of products by manufacturing places"
m

In [ ]:
for countries in (['MX', 'AU', 'RU', 'NL', 'SE', 'PT', 'LU', 'DK', 'RO', 'HU']):
    popup = country_focus_png(countries, 'manufacturing_places')
    marker = folium.Marker(coordinates_estimator(countries), popup = popup)
    m.add_child(marker)
m.add_child(linear)
m

In [ ]:
m.save(outfile= "manufacturing_places_km.html")